In [1]:
import cv2

In [8]:
def remove_borders(img):
    contours,hierarchy = cv2.findContours(img,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    cntsSorted = sorted(contours, key=lambda x: cv2.contourArea(x))
    cnt = cntsSorted[-1]
    print (len(contours))
    x,y,w,h = cv2.boundingRect(cnt)
    crop = img[y:y+h,x:x+w]
    return (crop)

In [9]:
img = cv2.imread("border.png")
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
_,img=cv2.threshold(img,180,255,cv2.THRESH_BINARY)
cv2.imwrite("thresholded.png",img)
img = cv2.imread("thresholded.png",cv2.IMREAD_UNCHANGED)
no_borders = remove_borders(img)
cv2.imwrite("no_borders.png", no_borders)
img = cv2.imread("no_borders.png",cv2.IMREAD_UNCHANGED)
no_borders = remove_borders(img)
cv2.imwrite("no_borders.png", no_borders)

324
156


True

# #Bounding Box

In [1]:
import cv2

In [14]:
image = cv2.imread("no_borders.png")
base_image = image.copy()
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray, (7,7), 0)
cv2.imwrite("index_blur.png", blur)
thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
cv2.imwrite("index_thresh.png", thresh)
kernal = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 16))
cv2.imwrite("index_kernal.png", kernal)

True

In [15]:
dilate = cv2.dilate(thresh, kernal, iterations=1)
cv2.imwrite("index_dilate.png", dilate)

True

In [16]:
cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

In [17]:
cnts = cnts[0] if len(cnts) == 2 else cents[1]

In [18]:
cnts = sorted(cnts, key=lambda x: cv2.boundingRect(x)[0])

In [19]:
for c in cnts:
    x, y, w, h = cv2.boundingRect(c)
    if h > 200 and w > 20:
        roi = image[y:y+h, x:x+h]
        cv2.imwrite("index_roi.png", roi)
        cv2.rectangle(image, (x, y), (x+w, y+h), (36, 255, 12), 2)
cv2.imwrite("index_bbox_new.png", image)

True

In [3]:
import cv2

image = cv2.imread('border.png')

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (4, 10))
detected_lines = cv2.morphologyEx(thresh, cv2.MORPH_OPEN,horizontal_kernel, iterations=2)

cnts = cv2.findContours(detected_lines, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]

for c in cnts:
    cv2.drawContours(image, [c], -1, (255, 255, 255), 2)

repair_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 16))

result = 255 - cv2.morphologyEx(255 - image, cv2.MORPH_CLOSE, repair_kernel,iterations=1)

cv2.imwrite("result_image.png", result)

True

In [2]:
import cv2

image = cv2.imread('border.png')
removed = image.copy()
gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

# Remove vertical lines
vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1,40))
remove_vertical = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, vertical_kernel, iterations=2)
cnts = cv2.findContours(remove_vertical, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
for c in cnts:
    cv2.drawContours(removed, [c], -1, (255,255,255), 15)

# Remove horizontal lines
horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (40,1))
remove_horizontal = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, horizontal_kernel, iterations=2)
cnts = cv2.findContours(remove_horizontal, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
for c in cnts:
    cv2.drawContours(removed, [c], -1, (255,255,255), 5)

# Repair kernel
repair_kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
removed = 255 - removed
dilate = cv2.dilate(removed, repair_kernel, iterations=5)
dilate = cv2.cvtColor(dilate, cv2.COLOR_BGR2GRAY)
pre_result = cv2.bitwise_and(dilate, thresh)

result = cv2.morphologyEx(pre_result, cv2.MORPH_CLOSE, repair_kernel, iterations=5)
final = cv2.bitwise_and(result, thresh)

invert_final = 255 - final

cv2.imwrite('thresh_new.png', thresh)
cv2.imwrite('removed_new.png', removed)
cv2.imwrite('dilate_new.png', dilate)
cv2.imwrite('pre_result_new.png', pre_result)
cv2.imwrite('result_new.png', result)
cv2.imwrite('final_new.png', final)
cv2.imwrite('invert_final_new.png', invert_final)

True

In [20]:
import cv2
import numpy as np

# load image
img = cv2.imread("border.png")
img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
# threshold
thresh = cv2.threshold(img, 180, 255,cv2.THRESH_BINARY)[1]

# apply open morphology
#kernel = np.ones((5,5), np.uint8)
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (10,15))
morph = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel,iterations=1)
 
# get bounding box coordinates from largest external contour
contours = cv2.findContours(morph, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours = contours[0] if len(contours) == 2 else contours[1]

big_contour = max(contours, key=cv2.contourArea)
x,y,w,h = cv2.boundingRect(big_contour)

# crop image contour image
result = img.copy()
result = result[y:y+h, x:x+w]

# write result to disk
cv2.imwrite("sonar_thresh.png", thresh)
cv2.imwrite("sonar_morph.png", morph)
cv2.imwrite("sonar_cropped.png", result)

for c in contours:
    x, y, w, h = cv2.boundingRect(c)
    if h > 200 and w > 20:
        roi = thresh[y:y+h, x:x+h]
        cv2.imwrite("roi.png", roi)
        cv2.rectangle(thresh, (x, y), (x+w, y+h), (36, 255, 12), 2)

In [39]:
import numpy as np
import argparse
import cv2

image = cv2.imread("border.png")
_,image=cv2.threshold(image,170,255,cv2.THRESH_BINARY)
#print(image.shape[0])
#cv2.imshow("Original", image)
# a mask is the same size as our image, but has only two pixel
# values, 0 and 255 -- pixels with a value of 0 (background) are
# ignored in the original image while mask pixels with a value of
# 255 (foreground) are allowed to be kept
mask = np.zeros(image.shape[:2], dtype="uint8")
rectangle=cv2.rectangle(mask, (120,100), (662, 982), 255, -1)
print(rectangle)
cv2.imwrite("RectangularMask.png", mask)
# apply our mask -- notice how only the person in the image iscropped out
masked = cv2.bitwise_and(image, image, mask=mask)
cv2.imwrite("MaskApplied.png", masked)
mask_inv=cv2.bitwise_not(mask)
cv2.imwrite("mask.png", mask_inv)
white_mask= cv2.bitwise_not(masked,mask=mask_inv)
cv2.imwrite("white_mask.png", white_mask)
white_masked= cv2.add(masked, white_mask)
cv2.imwrite("white_masked.png", white_masked)

out_image_array=[]
total_lines = 0
input_image=cv2.imread("white_masked.png")

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [42]:
# DESCRIPTION:
# The training images are thresholded to remove any backgrounf noise here.
import cv2
import os
import matplotlib.pyplot as plt
import time
import sys
import numpy as np
import matplotlib.pyplot as plt
import pdb
import scipy.io as sio
import numpy as np
import os
from PIL import Image

# Main function to perform the splitting..../////////////////////////////////////////////
def line_splitter(input_image):
    # Main list to store the output ligatures i.e. a list of numpy arrays.
    out_image_array=[]

    # Thresholding the images
 #   _, input_image = cv2.threshold(input_image, 5, 255, cv2.THRESH_BINARY)

    # Variable for storing the total number of lines.
    total_lines = 0

    # Extracting the dimensions of the image
    height,width=input_image.shape

    # Making a bool type filter
    filter=np.ones(shape=(1,width),dtype=bool)

    # Converting the source image to bool.
    input_image=input_image.astype(bool)


    # Some workplace variables for the following for loop.
    row=0   #This variable indexs the pixels of the image row wise.
    first_row=True
    line_detected=False

    # This for loop iterates through each pixel wide line of the page and separates them.
    for i in input_image:
        i=np.reshape(i,newshape=(1,width))
        res=np.bitwise_and(i,filter)
        res=np.bitwise_and.reduce(res,axis=1)

        if res==False:
            line_detected=True
            if first_row==True:
                row_start=row
                first_row=False
            row = row + 1
            continue

        if line_detected==True:
            row_end=row
            line_detected=False
            first_row = True
            out_image=input_image[row_start:row_end,:]
            out_image=out_image.astype(int)  #Converting back to int.
            out_image=out_image*255          #Replacing the 1's by 255's.
            # Writing the output.
            out_image_array.append(out_image)
            total_lines=total_lines+1
        row = row + 1
        print("Lines separated successfully. Total lines {}.".format(total_lines))
    # Returning the list..
    return out_image_array

# Testing the function/////////////////////////////////////////////////////////////////////////

# Selecting the output directory.
OUTDIR='border image lines'
if not os.path.isdir(OUTDIR):
    os.mkdir(OUTDIR)

# Selecting the source directory
IMDIR='border images'

# list to store the results.
list_of_lines=[]

# Making a counter to keep track of the images.
counter=0

# Function to sort images in the image directory
def file_sorter(element):
    e=int(element[:-4])
    return e

# The main for loop iterating through each image.
for image in sorted(os.listdir(IMDIR),key=file_sorter):
    digitized_image = cv2.imread(os.path.join(IMDIR, image), cv2.IMREAD_GRAYSCALE)
    list_of_lines=line_splitter(digitized_image)

    for line in list_of_lines:
        cv2.imwrite(os.path.join(OUTDIR, "{}.png".format(counter)), line)
        counter = counter + 1
    # Printing the result so far.
    sys.stdout.write("\rImages Left {}/{}.".format(image[:-4], len(os.listdir(IMDIR))))
    sys.stdout.flush()


Lines separated successfully. Total lines 0.
Lines separated successfully. Total lines 0.
Lines separated successfully. Total lines 0.
Lines separated successfully. Total lines 0.
Lines separated successfully. Total lines 0.
Lines separated successfully. Total lines 0.
Lines separated successfully. Total lines 0.
Lines separated successfully. Total lines 0.
Lines separated successfully. Total lines 0.
Lines separated successfully. Total lines 0.
Lines separated successfully. Total lines 0.
Lines separated successfully. Total lines 0.
Lines separated successfully. Total lines 0.
Lines separated successfully. Total lines 0.
Lines separated successfully. Total lines 0.
Lines separated successfully. Total lines 0.
Lines separated successfully. Total lines 0.
Lines separated successfully. Total lines 0.
Lines separated successfully. Total lines 0.
Lines separated successfully. Total lines 0.
Lines separated successfully. Total lines 0.
Lines separated successfully. Total lines 0.
Lines sepa